In [1]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


In [4]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean(numeric_only=True))
inputs

,NumRooms,Alley
0,3.0,Pave
1,2.0,NaN
2,4.0,NaN
3,3.0,NaN


In [5]:
inputs = pd.get_dummies(inputs, dummy_na=True)
inputs = inputs.astype('float32')
print(inputs)
print(inputs.dtypes)

   NumRooms  Alley_Pave  Alley_nan
0       3.0         1.0        0.0
1       2.0         0.0        1.0
2       4.0         0.0        1.0
3       3.0         0.0        1.0
NumRooms      float32
Alley_Pave    float32
Alley_nan     float32
dtype: object


In [6]:
import torch

X, y = torch.tensor(inputs.values, dtype=torch.float32), torch.tensor(outputs.values, dtype=torch.float32)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]]),
 tensor([127500., 106000., 178100., 140000.]))

# Thinking

In [7]:
import torch

# 1. 创建一个连续的 Tensor
x = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(f"原始张量是否连续: {x.is_contiguous()}")  # True

# --- 场景 A: 正常的 View ---
y = x.view(6) 
print("View 成功") # 成功，因为 x 是连续的

# --- 场景 B: 破坏连续性 ---
# 使用 transpose 转置，导致逻辑顺序变了，但物理内存没变
x_t = x.transpose(0, 1) 
print(f"转置后形状: {x_t.shape}")
print(f"转置后是否连续: {x_t.is_contiguous()}")  # False

# --- 场景 C: View 报错 ---
try:
    z = x_t.view(6)
except RuntimeError as e:
    print(f"\nView 报错: {e}") 
    # 报错内容: view size is not compatible with input tensor's meta data...

# --- 场景 D: Reshape 救场 ---
z = x_t.reshape(6)
print(f"\nReshape 成功: {z}") 
# reshape 在内部检测到 x_t 不连续，自动执行了 .contiguous() 然后再 view

原始张量是否连续: True
View 成功
转置后形状: torch.Size([3, 2])
转置后是否连续: False

View 报错: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

Reshape 成功: tensor([1, 4, 2, 5, 3, 6])


当只用 `赋值` 运算时，并不会分配内存空间

In [11]:
A = torch.arange(3)
B = A
print('id(A):',id(A),'id(B):', id(B))
B = A.clone()
print('id(A):',id(A),'id(B):', id(B))

id(A): 135411635707552 id(B): 135411635707552
id(A): 135411635707552 id(B): 135415914357440


In [12]:
A = torch.arange(3)
B = A
B += 2
A, B

(tensor([2, 3, 4]), tensor([2, 3, 4]))